<a href="https://colab.research.google.com/github/bryan-wolff/EA619R_2021S1/blob/main/Experimento%201/EA619_Experimento_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **EA619 - Laboratório de Análise Linear**
## Experimento 1 - Amostragem de Sinais
### Bryan Wolff - RA: 214095
### João Luís Carvalho de Abreu - RA: 175997



## **Importando Bibliotecas**

In [ ]:
!pip install pysoundfile

In [ ]:
'''Importando bibliotecas necessárias para execução geral do programa, 
trabalhos com áudio e para utilização de arquivos salvos no drive'''

#Geral
import math
import numpy as np
import matplotlib.pyplot as plt 
from scipy import signal
import plotly.graph_objects as go
from plotly.offline import iplot, init_notebook_mode

#Audio
from IPython.display import Audio
from scipy.io import wavfile
import cffi
import librosa
import IPython.display as ipd

#Drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

'''Necessário código de autenticação para acesso ao arquivo do drive.
Para isso, basta clicar no link, permitir acesso e copiar o código 
disponibilizado'''

In [ ]:
# Configurando o Plotly

def configure_plotly_browser_state():
    import IPython
    display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-1.5.1.min.js?noext',
            },
          });
        </script>
        '''))

## **Arquivo de Áudio Original**
Este item restringe-se apenas ao carregamento do arquivo de audio a ser analisado ao longo deste exercício.
Neste caso, utilizou-se o ***librosa***.

In [ ]:
#Leitura de arquivo de áudio do drive
audio_Y = drive.CreateFile({'id':'132hwAsOEAgDZNnd9rsfHi_h9WJvEN4QQ'})
audio_Y.GetContentFile('elefante.wav')
audio_Y, f_s = librosa.load('elefante.wav')

print('Taxa de Amostragem (fs): {} Hz\n'.format(f_s))

#Áudio original
ipd.Audio(audio_Y,rate=f_s)

## **Rotinas**

**Rotinas: Espectro e Plot**

In [ ]:
def espectro(func, fs):
    sinal = np.fft.fft(func) #transf. de Fourier
    w = np.linspace(0,fs,sinal.size) #frequencias avaliadas
    return w,sinal

def plot(w, sinal, yaxis):
    #no intuito de obter um gráfico interativo para a visualização da principal frequência de cada grupo, utilizou-se a biblioteca plotly
    configure_plotly_browser_state()
    init_notebook_mode(connected = False)

    fig = go.Figure(data=go.Scatter(x=w, 
                                    y=np.abs(sinal),
                                    mode='lines',
                                    line=dict(color='#fc3d03')),
                    layout=go.Layout(title='DFT do Sinal de Áudio',
                                     xaxis=dict(title='Freq. (Hz)'),
                                     yaxis=dict(title=yaxis),
                                     plot_bgcolor='#f2f0f0'))
    fig.show()


**Rotina de Subamostragem**

In [ ]:
def subamostragem(espectro, M):
    #rotina de subamostragem (decimação) que reduz a taxa de amostragem por um fator de M
    sub = []
    for i in range(0,len(espectro), M):
      sub.append(espectro[i])
    return sub

## **Questão 1**

Pelo Teorema de Nyquist, para possibilitar o registro digital de todas as frequências analisadas e poder recuperálas após a amostragem, é necessário que a taxa de amostragem ($f_s$) seja ao menos duas vezes maior que a frequência do sinal original ($f_m$), isto é, $f_s > 2f_m$. Logo, tendo em vista $f_s$ = 22050 Hz, $f_m$ = 11025 Hz.

## **Questão 2**

De acordo com o Teorema de Nyquist, quando a taxa de amostragem para determinado sinal é menor que a frequência de Nyquist, ocorre o efeito *aliasing*, em que uma alta frequência é medida erroneamente como sendo de frequência mais baixa, o que implica uma superposição dos espectros e a impossibilidade de recuperação do sinal analógico original.

In [ ]:
#Espectro do sinal de áudio original do elefante
w_Y, Y = espectro(audio_Y, f_s)
plot(w_Y, Y, '|Y|')

Ao aplicar a transformada de Fourier no sinal, é possível visualizar o espectro do aúdio a partir do gráfico gerado acima. Dado a exisistência de periodicidade em $f_s$, as frequências positivas e negativas podem ser verificadas através do eixo $\omega$ pela simetria do espectro. Assim, as componentes negativas são de 11025Hz a 22050Hz, enquanto as componentes positivas são de 0 a 11025 Hz.

É notável a existência de dois grupos de frequência relevantes na caracterização do sinal Y. Essas frequências podem ser verificadas a partir do cursor do gráfico interativo, no qual o primeiro grupo destaca-se uma frequência de aproximadamente 656Hz e o segundo grupo uma frequência de 1318Hz.

In [ ]:
#dado o fator M = 7
audio_X = subamostragem(audio_Y, 7) #subamostragem do sinal original

## **Questão 3**

Considerando que as amostras para x são tomadas de 7 em 7, é possível afirmar que a frequência de amostragem neste caso é, portanto, sete vezes menor que a do sinal original. Assim, $$f_s' = \frac{f_s}{7} = \frac{22050}{7} = 3150 Hz$$ 

In [ ]:
#A partir da rotina de espectro obtem-se o seguinte espectro do áudio decimado
w_X, X = espectro(audio_X, f_s/7)
plot(w_X, X, '|X|')

In [ ]:
ipd.Audio(audio_X, rate=f_s/7)

Logo ao escutar os dois áudios (original e subamostrado), é possível notar uma considerável perda na qualidade do áudio que passou pelo processo de decimação. Há uma sensação de que o som está abafado, mais grave e com notável ruído, características que podem ser apontadas como resultantes do efeito de *aliasing*.

## **Questão 4**

Tendo em vista o teorema de Nyquist já abordado na Questão 1, é possível afirmar que, para uma taxa de amostragem do sinal decimado equivalente a 3150 Hz, apenas as frequências até 1575 Hz serão preservadas este valor corresponde a 14ª parte da taxa de amostragem do sinal original (22050 Hz).

Nesse sentido, para eliminar o efeito de aliasing da maneira sugerida pelo enunciado da questão, é necessário zerar todas as frequências acima de 1575 Hz. Em outras palavras, ao dividir nosso espectro em 14 partes iguais, apenas a primeira e última parte serão preservadas, mais especificamente as partes referentes às freqûencia até 1575 Hz e a partir de 20475 Hz.


In [ ]:
aux = Y #cópia do sinal original a ser modificada

#zerando as componentes referentes às frequências fora do espectro a ser preservado
K_inicial = len(aux)//14
K_final = len(aux)-len(aux)//14
aux[K_inicial:K_final] = 0

audio_edit = np.fft.ifft(aux) #transformada inversa do sinal parcialmente zerado --> recuperar sinal original

# Espectro recuperado
w_edit, X_edit = espectro(audio_edit, f_s)
plot(w_edit, X_edit, 'Magnitude do Sinal Editado')

In [ ]:
# Subamostragem do sinal recuperado
audio_Z = subamostragem(audio_edit, 7)
w_Z, Z = espectro(audio_Z, f_s/7)
plot(w_Z, Z, '|Z|')

## **Questão 5**

Comparando os sinais Y, X e o sinal Z subamostrado, no qual foi computado logo abaixo, é possível notar que tanto o sinal X quanto o sinal Z tem perda considerável na qualidade do áudio em comparação com o original (Y). Porém, a qualidade do sinal Z, no qual foi aplicado a estratégia para evitar o **aliasing** é ligeiramente melhor que o sinal X.

In [ ]:
#@title **Áudio Y**
#Sinal orginial
ipd.Audio(audio_Y, rate=f_s)

In [ ]:
#@title **Áudio X**
#Sinal subamostrado
ipd.Audio(audio_X, rate=f_s/7)

In [ ]:
#@title **Áudio Z**
#Sinal parcialmente zerado e subamostrado
ipd.Audio(audio_Z, rate=f_s/7)